In [6]:
!pip install requests beautifulsoup4 tqdm
!pip install selenium webdriver-manager



[notice] A new release of pip is available: 23.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 9.7/9.7 MB 29.6 MB/s eta 0:00:00
     ---------------------------------------- 126.3/126.3 kB ? eta 0:00:00
     ------------------------------------- 481.7/481.7 kB 29.5 MB/s eta 0:00:00
     -------------------------------------- 157.7/157.7 kB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 45.8/45.8 kB ? eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB ? eta 0:00:00
     ---------------------------------------- 63.8/63.8 kB ? eta 0:00:00
INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 64.8/64.8 kB ? eta 0:00:00
     ---------------------------------------- 105.0/105.0 kB ? eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.0.1
    Uninstalling websocket-client-1.0.1:
      Successfully uninstalled webso

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.

[notice] A new release of pip is available: 23.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import json
import os
import string
import time
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Configuración Selenium
options = Options()
options.headless = True
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

BASE_URL = "https://www.drugs.com"
ALPHA_URL_TEMPLATE = "https://www.drugs.com/alpha/{}.html"
OUTPUT_FILE = "drugs_text_incremental.json"
SLEEP_TIME = 1.0  # Tiempo entre requests

# Cargar progreso si existe
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
        drug_texts = json.load(f)
    scraped_urls = {entry["url"] for entry in drug_texts}
    print(f"🔄 Progreso cargado: {len(drug_texts)} drogas ya scrapeadas.")
else:
    drug_texts = []
    scraped_urls = set()
    print("🆕 Comenzando desde cero.")

# Función para obtener links desde una página alpha
def get_links_from_alpha(pair):
    url = ALPHA_URL_TEMPLATE.format(pair)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    ul_tags = soup.find_all("ul", class_="ddc-list-column-2")
    links = []
    for ul in ul_tags:
        for li in ul.find_all("li"):
            a = li.find("a", href=True)
            if a:
                full_url = BASE_URL + a["href"]
                links.append((a.text.strip(), full_url))
    return links

# Función para extraer texto de una droga
def scrape_drug_text(name, url):
    try:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        content = soup.find("div", class_="contentBox") or soup.find("div", id="content")
        if not content:
            return None
        blocks = content.find_all(["p", "h2", "h3", "li"])
        text = "\n".join(b.get_text(strip=True) for b in blocks)
        return {"drug_name": name, "url": url, "text": text}
    except Exception as e:
        print(f"⚠️ Error en {url}: {e}")
        return None

# Recorrer todas las combinaciones de letras
combinaciones = [a + b for a in string.ascii_lowercase for b in string.ascii_lowercase]
procesadas = 0
batch_size = 10

for pair in tqdm(combinaciones):
    print(f"\n🔠 Procesando: {pair}")
    links = get_links_from_alpha(pair)
    print(f"🔗 {len(links)} links encontrados.")

    for name, url in links:
        if url in scraped_urls:
            continue

        print(f"🌐 Visitando: {name} ({url})")
        result = scrape_drug_text(name, url)
        if result:
            drug_texts.append(result)
            scraped_urls.add(url)
            print(f"✅ Guardado: {name}")
        else:
            print(f"⚠️ Sin contenido: {name}")

        procesadas += 1
        if procesadas % batch_size == 0:
            with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
                json.dump(drug_texts, f, ensure_ascii=False, indent=2)
            print(f"💾 Guardado parcial: {len(drug_texts)} drogas.")
        
        time.sleep(SLEEP_TIME)

# Guardado final
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(drug_texts, f, ensure_ascii=False, indent=2)
print(f"\n🎉 Finalizado. Total drogas guardadas: {len(drug_texts)}")

driver.quit()


🆕 Comenzando desde cero.



🔠 Procesando: aa
🔗 73 links encontrados.
🌐 Visitando: Abecma (https://www.drugs.com/abecma.html)
✅ Guardado: Abecma
🌐 Visitando: Abilify (https://www.drugs.com/abilify.html)
✅ Guardado: Abilify
🌐 Visitando: Abilify Asimtufii (https://www.drugs.com/abilify-asimtufii.html)
⚠️ Error en https://www.drugs.com/abilify-asimtufii.html: HTTPConnectionPool(host='localhost', port=55893): Read timed out. (read timeout=120)
⚠️ Sin contenido: Abilify Asimtufii



KeyboardInterrupt: 

In [9]:
import json
import string
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

BASE_URL = "https://www.drugs.com"
ALPHA_URL_TEMPLATE = "https://www.drugs.com/alpha/{}.html"
OUTPUT_FILE = "drug_links.json"

# Configuración Selenium
options = Options()
options.headless = True
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Función para extraer todos los links de una letra par (aa, ab, ..., zz)
def get_links_from_alpha(pair):
    url = ALPHA_URL_TEMPLATE.format(pair)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    ul_tags = soup.find_all("ul", class_="ddc-list-column-2")
    links = []
    for ul in ul_tags:
        for li in ul.find_all("li"):
            a = li.find("a", href=True)
            if a:
                full_url = BASE_URL + a["href"]
                links.append({"drug_name": a.text.strip(), "url": full_url})
    return links

# Recolectar todos los nombres y links
all_links = []
combinaciones = [a + b for a in string.ascii_lowercase for b in string.ascii_lowercase]

for pair in combinaciones:
    print(f"🔠 Procesando: {pair}")
    try:
        links = get_links_from_alpha(pair)
        all_links.extend(links)
        print(f"   ➕ {len(links)} links encontrados.")
    except Exception as e:
        print(f"⚠️ Error en {pair}: {e}")
    time.sleep(0.5)

# Guardar resultado en archivo
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_links, f, ensure_ascii=False, indent=2)

driver.quit()
print(f"\n✅ Finalizado. Total drogas: {len(all_links)} guardadas en {OUTPUT_FILE}")


🔠 Procesando: aa
   ➕ 73 links encontrados.
🔠 Procesando: ab
   ➕ 48 links encontrados.
🔠 Procesando: ac
   ➕ 184 links encontrados.
🔠 Procesando: ad
   ➕ 107 links encontrados.
🔠 Procesando: ae
   ➕ 0 links encontrados.
🔠 Procesando: af
   ➕ 42 links encontrados.
🔠 Procesando: ag
⚠️ Error en ag: HTTPConnectionPool(host='localhost', port=56793): Read timed out. (read timeout=120)
🔠 Procesando: ah
⚠️ Error en ah: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
🔠 Procesando: ai


KeyboardInterrupt: 

In [11]:
import json
import string
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://www.drugs.com"
ALPHA_URL_TEMPLATE = "https://www.drugs.com/alpha/{}.html"
OUTPUT_FILE = "drug_links.json"
OUTPUT_CSV = "drug_links.csv"

# Configuración Selenium
options = Options()
options.headless = True
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def get_links_from_alpha(pair):
    url = ALPHA_URL_TEMPLATE.format(pair)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    ul_tags = soup.find_all("ul", class_="ddc-list-column-2")
    links = []
    for ul in ul_tags:
        for li in ul.find_all("li"):
            a = li.find("a", href=True)
            if a:
                full_url = BASE_URL + a["href"]
                links.append({"drug_name": a.text.strip(), "url": full_url})
    return links

all_links = []
combinaciones = [a + b for a in string.ascii_lowercase for b in string.ascii_lowercase]

for i, pair in enumerate(combinaciones):
    print(f"🔠 Procesando: {pair}")
    try:
        links = get_links_from_alpha(pair)
        all_links.extend(links)
        print(f"   ➕ {len(links)} links encontrados.")
    except Exception as e:
        print(f"⚠️ Error en {pair}: {e}")
    time.sleep(0.5)

    # Guardar progresivamente cada 10 pares procesados
    if (i + 1) % 2 == 0:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(all_links, f, ensure_ascii=False, indent=2)
        df = pd.DataFrame(all_links)
        df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
        print(f"⚡ Guardado provisional después de procesar {i+1} pares")
        
        time.sleep(2.0)

# Guardar al finalizar (por si no terminó en múltiplo de 10)
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_links, f, ensure_ascii=False, indent=2)
df = pd.DataFrame(all_links)
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

driver.quit()
print(f"\n✅ Finalizado. Total drogas: {len(all_links)} guardadas en {OUTPUT_FILE} y en tabla {OUTPUT_CSV}")


🔠 Procesando: aa
   ➕ 73 links encontrados.
🔠 Procesando: ab
   ➕ 48 links encontrados.
⚡ Guardado provisional después de procesar 2 pares
🔠 Procesando: ac
   ➕ 184 links encontrados.
🔠 Procesando: ad
   ➕ 107 links encontrados.
⚡ Guardado provisional después de procesar 4 pares
🔠 Procesando: ae
   ➕ 0 links encontrados.
🔠 Procesando: af
   ➕ 42 links encontrados.
⚡ Guardado provisional después de procesar 6 pares
🔠 Procesando: ag
   ➕ 0 links encontrados.
🔠 Procesando: ah
   ➕ 0 links encontrados.
⚡ Guardado provisional después de procesar 8 pares
🔠 Procesando: ai
   ➕ 0 links encontrados.
🔠 Procesando: aj
   ➕ 0 links encontrados.
⚡ Guardado provisional después de procesar 10 pares
🔠 Procesando: ak
   ➕ 25 links encontrados.
🔠 Procesando: al
⚠️ Error en al: HTTPConnectionPool(host='localhost', port=57539): Read timed out. (read timeout=120)
⚡ Guardado provisional después de procesar 12 pares
🔠 Procesando: am
⚠️ Error en am: HTTPConnectionPool(host='localhost', port=57539): Read timed

KeyboardInterrupt: 

In [12]:
# Acá armo uno para ver si puedo agarrar las que son columna única, las que NO tienen, y evitar los timeouts.


In [14]:
import json
import string
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException
from bs4 import BeautifulSoup

# Constantes
BASE_URL = "https://www.drugs.com"
ALPHA_URL_TEMPLATE = "https://www.drugs.com/alpha/{}.html"
OUTPUT_FILE = "drug_links.json"
OUTPUT_CSV = "drug_links.csv"
ERROR_LOG = "error_log.txt"

# Setup del navegador con user-agent real
def create_driver():
    options = Options()
    options.headless = True
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/114.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# Inicializar driver por primera vez
driver = create_driver()

# Scraper de una combinación
def get_links_from_alpha(pair):
    global driver
    try:
        url = ALPHA_URL_TEMPLATE.format(pair)
        driver.get(url)
        time.sleep(random.uniform(5, 9))  # Delay aleatorio

        soup = BeautifulSoup(driver.page_source, "html.parser")
        links = []

        # Buscar todas las listas relevantes
        ul_tags = soup.find_all("ul")
        for ul in ul_tags:
            for li in ul.find_all("li"):
                a = li.find("a", href=True)
                if a and a["href"].startswith("/"):
                    full_url = BASE_URL + a["href"]
                    links.append({"drug_name": a.text.strip(), "url": full_url})
        return links
    except Exception as e:
        raise e  # Se captura en el loop principal

# Loop principal
all_links = []
errores = []

#ACÁ PARA PONER DE DONDE ARRANCAR
START_FROM = "ai"

full_combinaciones = [a + b for a in string.ascii_lowercase for b in string.ascii_lowercase]
start_index = full_combinaciones.index(START_FROM)
combinaciones = full_combinaciones[start_index:]


for i, pair in enumerate(combinaciones):
    print(f"🔠 Procesando: {pair}")
    try:
        links = get_links_from_alpha(pair)
        all_links.extend(links)
        print(f"   ➕ {len(links)} links encontrados.")
    except Exception as e:
        print(f"⚠️ Error en {pair}: {e}")
        errores.append(pair)

        # Reintentar reiniciando el driver
        try:
            driver.quit()
        except:
            pass  # Por si ya está muerto
        time.sleep(5)
        driver = create_driver()

    # Guardado progresivo cada 10 combinaciones
    if (i + 1) % 2 == 0:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(all_links, f, ensure_ascii=False, indent=2)
        pd.DataFrame(all_links).to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
        print(f"💾 Guardado provisional luego de {i+1} pares")
        time.sleep(random.uniform(1, 3))

# Guardado final
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_links, f, ensure_ascii=False, indent=2)
pd.DataFrame(all_links).to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

# Log de errores
if errores:
    with open(ERROR_LOG, "w") as log:
        for e in errores:
            log.write(f"{e}\n")
    print(f"📝 Se registraron errores en {len(errores)} combinaciones. Ver {ERROR_LOG}")

# Cierre
driver.quit()
print(f"\n✅ Finalizado. Total drogas: {len(all_links)} guardadas en {OUTPUT_FILE} y {OUTPUT_CSV}")


🔠 Procesando: aa
   ➕ 172 links encontrados.
🔠 Procesando: ab
   ➕ 122 links encontrados.
💾 Guardado provisional luego de 2 pares
🔠 Procesando: ac
   ➕ 258 links encontrados.
🔠 Procesando: ad
   ➕ 181 links encontrados.
💾 Guardado provisional luego de 4 pares
🔠 Procesando: ae
   ➕ 55 links encontrados.
🔠 Procesando: af
   ➕ 116 links encontrados.
💾 Guardado provisional luego de 6 pares
🔠 Procesando: ag
   ➕ 60 links encontrados.
🔠 Procesando: ah
   ➕ 54 links encontrados.
💾 Guardado provisional luego de 8 pares
🔠 Procesando: ai
   ➕ 60 links encontrados.
🔠 Procesando: aj
⚠️ Error en aj: HTTPConnectionPool(host='localhost', port=59500): Read timed out. (read timeout=120)


KeyboardInterrupt: 

In [15]:
# Cambio para no tener los links al pedo (de a, b, c, etc.)

In [17]:
import json
import string
import time
import random
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException
from bs4 import BeautifulSoup

# Constantes
BASE_URL = "https://www.drugs.com"
ALPHA_URL_TEMPLATE = "https://www.drugs.com/alpha/{}.html"
OUTPUT_FILE = "drug_links.json"
OUTPUT_CSV = "drug_links.csv"
ERROR_LOG = "error_log.txt"

# Cargar datos previos si existen
all_links = []
urls_existentes = set()
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
        all_links = json.load(f)
        urls_existentes = set(link["url"] for link in all_links)
    print(f"📂 Cargados {len(all_links)} links previos desde {OUTPUT_FILE}")

# Setup del navegador con user-agent real
def create_driver():
    options = Options()
    options.headless = True
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/114.0.0.0 Safari/537.36")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Inicializar driver
driver = create_driver()

# Scraper de una combinación
def get_links_from_alpha(pair):
    global driver
    try:
        url = ALPHA_URL_TEMPLATE.format(pair)
        driver.get(url)
        time.sleep(random.uniform(5, 9))

        soup = BeautifulSoup(driver.page_source, "html.parser")
        links = []

        content = soup.find("div", id="content")
        ul_tags = content.find_all("ul") if content else []

        for ul in ul_tags:
            for li in ul.find_all("li"):
                a = li.find("a", href=True)
                if a and a["href"].startswith("/"):
                    full_url = BASE_URL + a["href"]
                    links.append({"drug_name": a.text.strip(), "url": full_url})
        return links
    except Exception as e:
        raise e

# Loop principal
errores = []
START_FROM = "ab"

full_combinaciones = [a + b for a in string.ascii_lowercase for b in string.ascii_lowercase]
start_index = full_combinaciones.index(START_FROM)
combinaciones = full_combinaciones[start_index:]

for i, pair in enumerate(combinaciones):
    print(f"🔠 Procesando: {pair}")
    try:
        links = get_links_from_alpha(pair)

        # Filtrar duplicados
        nuevos_links = [link for link in links if link["url"] not in urls_existentes]
        all_links.extend(nuevos_links)
        urls_existentes.update(link["url"] for link in nuevos_links)

        print(f"   ➕ {len(nuevos_links)} nuevos links encontrados.")
    except Exception as e:
        print(f"⚠️ Error en {pair}: {e}")
        errores.append(pair)

        try:
            driver.quit()
        except:
            pass
        time.sleep(random.uniform(3, 7))
        driver = create_driver()

    # Guardado progresivo cada 2 combinaciones
    if (i + 1) % 2 == 0:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(all_links, f, ensure_ascii=False, indent=2)
        pd.DataFrame(all_links).to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
        print(f"💾 Guardado provisional luego de {i + 1} pares")
        time.sleep(random.uniform(1, 3))

# Guardado final
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_links, f, ensure_ascii=False, indent=2)
pd.DataFrame(all_links).to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

if errores:
    with open(ERROR_LOG, "w") as log:
        for e in errores:
            log.write(f"{e}\n")
    print(f"📝 Se registraron errores en {len(errores)} combinaciones. Ver {ERROR_LOG}")

driver.quit()
print(f"\n✅ Finalizado. Total drogas: {len(all_links)} guardadas en {OUTPUT_FILE} y {OUTPUT_CSV}")


🔠 Procesando: ab
   ➕ 72 nuevos links encontrados.
🔠 Procesando: ac
   ➕ 186 nuevos links encontrados.
💾 Guardado provisional luego de 2 pares
🔠 Procesando: ad
   ➕ 108 nuevos links encontrados.
🔠 Procesando: ae
   ➕ 5 nuevos links encontrados.
💾 Guardado provisional luego de 4 pares
🔠 Procesando: af
   ➕ 43 nuevos links encontrados.
🔠 Procesando: ag
   ➕ 10 nuevos links encontrados.
💾 Guardado provisional luego de 6 pares
🔠 Procesando: ah
   ➕ 4 nuevos links encontrados.
🔠 Procesando: ai
   ➕ 10 nuevos links encontrados.
💾 Guardado provisional luego de 8 pares
🔠 Procesando: aj
   ➕ 3 nuevos links encontrados.
🔠 Procesando: ak
   ➕ 26 nuevos links encontrados.
💾 Guardado provisional luego de 10 pares
🔠 Procesando: al
   ➕ 310 nuevos links encontrados.
🔠 Procesando: am
   ➕ 188 nuevos links encontrados.
💾 Guardado provisional luego de 12 pares
🔠 Procesando: an
   ➕ 151 nuevos links encontrados.
🔠 Procesando: ao
   ➕ 32 nuevos links encontrados.
💾 Guardado provisional luego de 14 pares
🔠